# 🔍 Multi-Drift Monitoring with DriftWatch

This notebook demonstrates how to detect **Feature Drift**, **Prediction Drift**, and **Concept Drift** using DriftWatch's unified monitoring suite.

## What You'll Learn

1. The difference between the 3 types of drift
2. How to use each individual monitor
3. How to combine them with `DriftSuite`
4. How to interpret `ComprehensiveDriftReport` results

---

## 📦 Installation

```bash
pip install driftwatch
```

In [ ]:
import numpy as np
import pandas as pd

from driftwatch import (
    ComprehensiveDriftReport,
    ConceptMonitor,
    DriftSuite,
    DriftType,
    Monitor,
    PredictionMonitor,
)

print(f"DriftWatch version: {__import__('driftwatch').__version__}")

## 🏦 Scenario: Credit Scoring Model

We'll simulate a credit scoring model that predicts loan defaults.

**Training period**: Normal economic conditions
**Production period**: Economic downturn → applicants have different profiles

In [ ]:
def generate_credit_data(
    n: int,
    seed: int,
    age_mean: float = 38.0,
    income_mean: float = 55000.0,
    debt_ratio_mean: float = 0.35,
):
    """Generate realistic credit scoring data."""
    rng = np.random.default_rng(seed)

    # Features
    age = rng.normal(age_mean, 12, n).clip(18, 80)
    income = rng.lognormal(np.log(income_mean), 0.6, n).clip(15000, 500000)
    debt = rng.beta(2, 4, n) * debt_ratio_mean * 3
    history = rng.poisson(72, n).clip(0, 360)
    credit_lines = rng.poisson(4, n).clip(0, 20)
    employment = rng.exponential(6, n).clip(0, 40)

    features = pd.DataFrame(
        {
            "age": age,
            "annual_income": income,
            "debt_ratio": debt,
            "credit_history_months": history,
            "num_credit_lines": credit_lines,
            "employment_years": employment,
        }
    )

    # Labels & predictions
    log_odds = (
        -2.0
        + 0.01 * (age - 40)
        - 0.00002 * (income - 50000)
        + 3.0 * (debt - 0.3)
        - 0.005 * (history - 60)
        + rng.normal(0, 0.5, n)
    )
    prob = 1 / (1 + np.exp(-log_odds))
    y_true = (rng.random(n) < prob).astype(int)

    pred_prob = 1 / (1 + np.exp(-(log_odds + rng.normal(0, 0.3, n))))
    y_pred = (pred_prob > 0.5).astype(int)

    return features, y_true, y_pred


# Reference data (training conditions)
X_ref, y_ref_true, y_ref_pred = generate_credit_data(n=2000, seed=42)

# Production data (normal conditions)
X_prod_ok, y_prod_ok_true, y_prod_ok_pred = generate_credit_data(n=800, seed=99)

# Production data (economic downturn — DRIFTED)
X_prod_drift, y_prod_drift_true, y_prod_drift_pred = generate_credit_data(
    n=800,
    seed=77,
    age_mean=48.0,
    income_mean=35000.0,
    debt_ratio_mean=0.55,
)

print(f"Reference: {X_ref.shape}")
print(f"Production OK: {X_prod_ok.shape}")
print(f"Production Drift: {X_prod_drift.shape}")

---

## 📊 Feature Drift (P(X) — Data Distribution)

The most basic type: are the input features still distributed the same way?

In [ ]:
feature_monitor = Monitor(reference_data=X_ref)

# ✅ No drift expected
report_ok = feature_monitor.check(X_prod_ok)
print("=== Normal Production ===")
print(f"Has drift: {report_ok.has_drift()}")
print(f"Drift ratio: {report_ok.drift_ratio():.1%}")
print(f"Drifted features: {report_ok.drifted_features()}")
print()

# ⚠️ Drift expected (economic downturn)
report_drift = feature_monitor.check(X_prod_drift)
print("=== Downturn Production ===")
print(f"Has drift: {report_drift.has_drift()}")
print(f"Drift ratio: {report_drift.drift_ratio():.1%}")
print(f"Drifted features: {report_drift.drifted_features()}")

## 🎯 Prediction Drift (P(Ŷ) — Model Output Distribution)

Are the predictions still distributed the same way? No labels needed!

In [ ]:
pred_monitor = PredictionMonitor(
    reference_predictions=y_ref_pred,
    detector="psi",
)

# ✅ Normal conditions
report_pred_ok = pred_monitor.check(y_prod_ok_pred)
print("=== Normal Production ===")
print(f"Prediction drift: {report_pred_ok.has_drift()}")
for r in report_pred_ok.feature_results:
    print(f"  {r.feature_name}: {r.method}={r.score:.4f} (drift={r.has_drift})")
    print(f"  Drift type: {r.drift_type.value}")
print()

# ⚠️ Drifted conditions
report_pred_drift = pred_monitor.check(y_prod_drift_pred)
print("=== Downturn Production ===")
print(f"Prediction drift: {report_pred_drift.has_drift()}")
for r in report_pred_drift.feature_results:
    print(f"  {r.feature_name}: {r.method}={r.score:.4f} (drift={r.has_drift})")

## 🧠 Concept Drift (P(Y|X) — Performance Degradation)

Has the model's accuracy degraded? Requires ground truth labels.

In [ ]:
concept_monitor = ConceptMonitor(
    task="classification",
    metrics=["accuracy", "f1", "precision", "recall"],
)

# Check concept drift
concept_report = concept_monitor.check(
    y_true_ref=y_ref_true,
    y_pred_ref=y_ref_pred,
    y_true_prod=y_prod_drift_true,
    y_pred_prod=y_prod_drift_pred,
)

print(f"Concept drift detected: {concept_report.has_drift()}")
print()

# Show performance details
for detail in concept_monitor.performance_details:
    print(
        f"{detail.metric_name:>12}: "
        f"ref={detail.reference_value:.3f} → "
        f"prod={detail.production_value:.3f} "
        f"(Δ{detail.absolute_change:+.3f}) "
        f"{'⚠ DEGRADED' if detail.has_degradation else '✓ OK'}"
    )

---

## 🔗 Unified Monitoring with DriftSuite

Combine all three drift types in a single check!

In [ ]:
suite = DriftSuite(
    reference_data=X_ref,
    reference_predictions=y_ref_pred,
    task="classification",
    performance_metrics=["accuracy", "f1"],
    model_version="credit-v1.0",
)

# Full comprehensive check
report = suite.check(
    production_data=X_prod_drift,
    production_predictions=y_prod_drift_pred,
    y_true_ref=y_ref_true,
    y_pred_ref=y_ref_pred,
    y_true_prod=y_prod_drift_true,
    y_pred_prod=y_prod_drift_pred,
)

print(report.summary())

In [ ]:
# Programmatic access
print(f"Overall status: {report.status.value}")
print(f"Drift types detected: {[d.value for d in report.drift_types_detected()]}")
print(f"Model version: {report.model_version}")
print()

# Feature drift details
if report.feature_report:
    print(f"Feature drift ratio: {report.feature_report.drift_ratio():.1%}")
    print(f"Drifted features: {report.feature_report.drifted_features()}")
print()

# Prediction drift details
if report.prediction_report:
    print(f"Prediction drift: {report.prediction_report.has_drift()}")
print()

# Concept drift details
if report.concept_report:
    print(f"Concept drift: {report.concept_report.has_drift()}")

## 📤 Serialization

Export results for logging, dashboards, or alerting.

In [ ]:
# As dictionary
report_dict = report.to_dict()
print("Keys:", list(report_dict.keys()))
print(f"Detected types: {report_dict['drift_types_detected']}")
print()

# As JSON
report_json = report.to_json(indent=2)
print(report_json[:500], "...")

## 🏭 Production Pipeline Pattern

Here's how to integrate multi-drift monitoring in a production pipeline:

In [ ]:
def monitor_batch(
    suite: DriftSuite,
    X_batch: pd.DataFrame,
    y_pred_batch: np.ndarray,
    y_true_batch: np.ndarray | None = None,
    y_ref_true: np.ndarray | None = None,
    y_ref_pred: np.ndarray | None = None,
) -> ComprehensiveDriftReport:
    """Monitor a production batch for all drift types."""
    report = suite.check(
        production_data=X_batch,
        production_predictions=y_pred_batch,
        y_true_ref=y_ref_true,
        y_pred_ref=y_ref_pred,
        y_true_prod=y_true_batch,
        y_pred_prod=y_pred_batch,
    )

    # Alerting logic
    if DriftType.CONCEPT in report.drift_types_detected():
        print("🚨 CRITICAL: Concept drift — model retraining required!")
    elif DriftType.PREDICTION in report.drift_types_detected():
        print("⚠️ WARNING: Prediction drift — investigate model outputs")
    elif DriftType.FEATURE in report.drift_types_detected():
        print("📊 INFO: Feature drift — monitor closely")
    else:
        print("✅ OK: No drift detected")

    return report


# Simulate batches
print("Batch 1 (normal):")
monitor_batch(suite, X_prod_ok, y_prod_ok_pred)

print("\nBatch 2 (economic downturn):")
monitor_batch(
    suite,
    X_prod_drift,
    y_prod_drift_pred,
    y_true_batch=y_prod_drift_true,
    y_ref_true=y_ref_true,
    y_ref_pred=y_ref_pred,
)

---

## 🏠 Bonus: Regression Example (House Pricing)

Multi-drift monitoring also works for regression tasks.

In [ ]:
# Generate house pricing data
rng = np.random.default_rng(42)
n = 1000

area = rng.lognormal(np.log(120), 0.4, n).clip(30, 500)
rooms = rng.poisson(3, n).clip(1, 10)
distance = rng.exponential(5, n).clip(0.5, 30)

X_house_ref = pd.DataFrame(
    {"area_sqm": area, "num_rooms": rooms, "distance_km": distance}
)
y_house_ref = 50000 + 2500 * area + 8000 * rooms - 3000 * distance
y_house_ref_pred = y_house_ref + rng.normal(0, 15000, n)

# Market boom → bigger houses, higher prices
area_boom = rng.lognormal(np.log(160), 0.4, n).clip(30, 500)
X_house_prod = pd.DataFrame(
    {
        "area_sqm": area_boom,
        "num_rooms": rng.poisson(4, n).clip(1, 10),
        "distance_km": rng.exponential(4, n).clip(0.5, 30),
    }
)
y_house_prod = (
    50000
    + 2500 * area_boom
    + 8000 * X_house_prod["num_rooms"].values
    - 3000 * X_house_prod["distance_km"].values
) * 1.4  # 40% price increase
y_house_prod_pred = y_house_prod + rng.normal(0, 30000, n)

# Regression DriftSuite
regression_suite = DriftSuite(
    reference_data=X_house_ref,
    reference_predictions=y_house_ref_pred,
    task="regression",
    performance_metrics=["rmse", "r2", "mae"],
    model_version="pricing-v2.0",
)

reg_report = regression_suite.check(
    production_data=X_house_prod,
    production_predictions=y_house_prod_pred,
    y_true_ref=y_house_ref,
    y_pred_ref=y_house_ref_pred,
    y_true_prod=y_house_prod,
    y_pred_prod=y_house_prod_pred,
)

print(reg_report.summary())

---

## Summary

| Drift Type | Class | Needs Labels? | Key Insight |
|------------|-------|:---:|-------------|
| Feature | `Monitor` | ❌ | Data distribution changed |
| Prediction | `PredictionMonitor` | ❌ | Model outputs changed |
| Concept | `ConceptMonitor` | ✅ | Model accuracy degraded |
| All-in-one | `DriftSuite` | Optional | Unified monitoring |

**Key takeaways:**
- Start with **Feature Drift** (always available, no labels needed)
- Add **Prediction Drift** for early warning (still label-free)
- Use **Concept Drift** when ground truth arrives (most actionable)
- Use `DriftSuite` to combine all three in production